In [ ]:
import pandas as pd
from Cross import Cross_section,cables, expansion_calc
from bridge_specs import box

In [ ]:
df=pd.read_excel('data/span.xlsx',index_col=None,header=None)
span = int(df.iloc[2])
no = int(df.iloc[1])
sc=[(j)/(no-1)*50 for j in range(no)]

***required:***
1. list of sections with cable profile
2. BM due to permanent dead load
3. BM due to superimposed dead load (without surfacing)
4. BM due to surfacing
5. BM due to live load (max of all vehicle loads)

**defining sections**

main section

In [ ]:

"""INPUTS SECTION"""

"""FOR CROSS SECTION"""
length=[]
height=[]


#####################################################################################################################

"""GUI INPUT OF BRIDGE SECTION LENGTHS AND HEIGHTS"""

if False:
    allinput()

df=pd.read_excel('data/box.xlsx',index_col=None,header=None)


height=df.values.tolist()[0]
length=df.values.tolist()[1]


section=Cross_section(length,height)
cable = cables(35,14,section)

Generating intermediate sections

In [ ]:

sections = []
for i in range(no):  
    section1=Cross_section(length,height)
    section1.expansion_width=expansion_calc(span,sc[i],cable)
    section1.cableprop=cable.cablepos(sc[i],50)
    sections.append(section1)

**Importing load**

In [ ]:
df = pd.read_excel('outputs/Moments.xlsx', index_col=0)
bm_pdl = list(df['Dead Load'])
bm_odl = list(df['Other Loads'])
bm_sidl = list(df['Surface Loads'])
bm_pedl = list(df['Pedestrian Load'])

df1 = pd.read_excel('outputs/loads.xlsx', index_col=[0, 1]) #live load
bm_ll = df1.loc['MaxBM'].max().to_list()

### Cable profile

In [ ]:
# def cable_profile(end_cable_pos, mid_cable_pos, span_length, division=8):
#     cable_profile = []
#     x = [(j)/division*span_length for j in range(division+1)]
#     for i in x:
#         cable_pos_x = []
#         for j in range(len(end_cable_pos)):
#             h = end_cable_pos[j][1]-mid_cable_pos[j][1]
#             a = end_cable_pos[j][1]
#             y = a + 4*h*i/span_length - 4*h*i**2/span_length**2
#             cable_pos_x.append((end_cable_pos[j][0], y))
#         cable_profile.append(cable_pos_x)
#     return cable_profile

### Eccentricity of cables

In [ ]:
def eccentricity(cable_positions, centroid):
    cable_areaxpos_sum = 0
    for i in cable_positions:
        cable_areaxpos_sum+=(centroid-i[1])
    return cable_areaxpos_sum/len(cable_positions)

### Stress

In [ ]:
list_of_sections = sections #dummy
prestressing_force_full = 3400 #dummy
prestressing_force_withloss = 0.75*prestressing_force_full

In [ ]:
def tau(section, bm):
    tau_top = bm*section.ymin/section.I[0]
    tau_bot = bm*section.ymax/section.I[0]
    return (tau_top, tau_bot)

**Case 1: Rare combination**

presetressing stress

In [ ]:
bm_pre = [prestressing_force_full*eccentricity(i.cableprop, i.Centroid[1]) for i in list_of_sections]

list_tau_pre = [tau(list_of_sections[index], bm_pre[index]) for index in range(len(list_of_sections))]

# for index, i in enumerate(list_of_sections):
#     list_tau_pre.append(tau(i, bm_pre[index]))
    
# list_tau_ax = [prestressing_force_full/sum(i.section_area) for i in list_of_sections]

list_tau_preTotal = [(list_tau_pre[index][0] + list_tau_ax[index], list_tau_pre[index][1] + list_tau_ax[index]) for index in range(len(list_of_sections))]

stress due to ded load

In [ ]:
list_tau_DL = [tau(list_of_sections[index], bm_pdl[index]) for index in range(len(list_of_sections))]

stress due to surfacing

In [ ]:
list_tau_sidl = [tau(list_of_sections[index], bm_sidl[index]) for index in range(len(list_of_sections))]

resultant stress

In [ ]:
list_tau_resultant = [(list_tau_preTotal[index][0] + list_tau_DL[index][0] + list_tau_ODL[index][0] + list_tau_sidl[index][0],
                       list_tau_preTotal[index][1] + list_tau_DL[index][1] + list_tau_ODL[index][1] + list_tau_sidl[index][1])
                      for index in range(len(list_of_sections))]
list_tau_resultant_factored = [(list_tau_preTotal[index][0] + list_tau_DL[index][0] + list_tau_ODL[index][0] + 1.2*list_tau_sidl[index][0],
                       list_tau_preTotal[index][1] + list_tau_DL[index][1] + list_tau_ODL[index][1] + 1.2*list_tau_sidl[index][1])
                      for index in range(len(list_of_sections))]

check (*left blank for now*)

**Case 2: Quasi Permanent Combination**

presetressing stress

In [ ]:
bm_pre = [prestressing_force_withloss*eccentricity(i.cableprop, i.Centroid[1]) for i in list_of_sections]

list_tau_pre = [tau(list_of_sections[index], bm_pre[index]) for index in range(len(list_of_sections))]
  
list_tau_ax = [prestressing_force_withloss/sum(i.section_area) for i in list_of_sections]

list_tau_preTotal = [(list_tau_pre[index][0] + list_tau_ax[index], list_tau_pre[index][1] + list_tau_ax[index]) for index in range(len(list_of_sections))]

stress due to other load

In [ ]:
list_tau_ODL = [tau(list_of_sections[index], bm_odl[index]) for index in range(len(list_of_sections))]

stress due to live load

In [ ]:
list_tau_LL = [tau(list_of_sections[index], bm_ll[index]) for index in range(len(list_of_sections))]

resultant

In [ ]:
list_tau_resultant = [(list_tau_preTotal[index][0] + list_tau_DL[index][0]+ list_tau_SIDL[index][0]+ list_tau_ODL[index][0] + list_tau_LL[index][0],
                       list_tau_preTotal[index][1] + list_tau_DL[index][1]+ list_tau_SIDL[index][1]+ list_tau_ODL[index][1] + list_tau_LL[index][1])
                      for index in range(len(list_of_sections))]

list_tau_resultant_factored = [(list_tau_preTotal[index][0] + list_tau_DL[index][0]+ 1.2*list_tau_SIDL[index][0]+ list_tau_ODL[index][0] + list_tau_LL[index][0],
                       list_tau_preTotal[index][1] + list_tau_DL[index][1]+ list_tau_SIDL[index][1]+ list_tau_ODL[index][1] + list_tau_LL[index][1])
                      for index in range(len(list_of_sections))]

check (*left blank for now*)